In [19]:
import numpy as np
import pandas as pd
import scipy.io
import time
import timeit

In [21]:
#------------------    Read in and Process Real Data    ----------------------#
# Read 311 data
t1 = time.time()
requests = pd.read_csv('../../311__Service_Requests.csv')
t2 = time.time()
print "Read in data in %.2f seconds." % (t2 - t1)

# Filter for closed cases
t1 = time.time()
closed_requests = requests[requests['CASE_STATUS'] == 'Closed']
t2 = time.time()
print "Filtered data in %.2f seconds." % (t2 - t1)

# Convert dates to datetime type
t1 = time.time()
for col in ['OPEN_DT', 'TARGET_DT', 'CLOSED_DT']:
    closed_requests[col] = pd.to_datetime(closed_requests[col], infer_datetime_format=True)
t2= time.time()
print "Dates processed in %.2f seconds." % (t2 - t1)

Read in data in 10.75 seconds.
Filtered data in 0.30 seconds.

/Users/isadoranun/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/isadoranun/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



Dates processed in 461.30 seconds.


In [31]:
#---------------- Plot Cluster Membership Histogram ----------------#

def filter_by_id(ids, source):
    begin = pd.to_datetime('January 01, 2015 12:00AM') 
    end = pd.to_datetime('January 01, 2016 12:00AM')

    # Filter for requests opened between begin and end dates
    in_range = closed_requests[closed_requests['OPEN_DT'] > begin]
    in_range = in_range[in_range['OPEN_DT'] < end]

    # Filter for requests with specific source
    in_range = in_range[in_range['Source'] == source]
    
   
    # Filter for requests for a set of case ids
    in_range = in_range[in_range['CASE_ENQUIRY_ID'].isin(ids)]
    
    return in_range

In [32]:
clusters = scipy.io.loadmat("2015_data_const_MAP_ids.mat")

In [33]:
ids0 = clusters['0'].tolist()[0]
ids1 = clusters['1'].tolist()[0]
ids2 = clusters['2'].tolist()[0]

In [42]:
a0 = filter_by_id(ids0,'Constituent Call')
a0 = a0[['LONGITUDE','LATITUDE']]
a0['Cluster'] = np.zeros(len(a0))
a1 = filter_by_id(ids1,'Constituent Call')
a1 = a1[['LONGITUDE','LATITUDE']]
a1['Cluster'] = np.ones(len(a1))
a2 = filter_by_id(ids2,'Constituent Call')
a2 = a2[['LONGITUDE','LATITUDE']]
a2['Cluster'] = 2*np.ones(len(a2))

In [58]:
all_data = pd.concat([a1,a0,a2])

In [59]:
all_data

,LONGITUDE,LATITUDE,Cluster
15,-71.0660,42.3590,1
35,-71.0694,42.3593,1
117,-71.0281,42.3381,1
160,-71.0891,42.3500,1
177,-71.0303,42.3813,1
205,-71.0683,42.3592,1
226,-71.0782,42.3367,1
265,-71.0345,42.3325,1
281,-71.0545,42.3342,1
302,-71.0791,42.3374,1


In [60]:
all_data = all_data.reset_index()

In [61]:
all_data.to_json("clusters.json", orient="records")

In [57]:
print len(a0),len(a1),len(a2)

2398 17894 114
